## Machine Learning and prediction
Elements of Data Science   
In this laboratory we will use training data to predict outcomes. We will first test these ideas using our Old Faithful data again. Next we will look at data on the iris flower to classify iris' based on sepal width and length. In our culminating activity we will predict molecular acidity using data computed by [Prof. Vince Voelz](http://www.voelzlab.org) in the Temple Chemistry department and a graduate student, Robert Raddi. See their paper: [Stacking Gaussian processes to improve pKa predictions in the SAMPL7 challenge](https://link.springer.com/epdf/10.1007/s10822-021-00411-8?sharing_token=yLV8dMXdxg40M_Ds_2Rhsfe4RwlQNchNByi7wbcMAY6fCl3bMLQiAhJzS2zZw-SwUkz490heLLZu1bPJ8T5LHXo1WvZkp0AJmWzXo71rszl8UaPxjqtqR-oARfxWGrTiCV0rNXy0C7IVzX6yoMYTPv2ZJfnQS-zF1pYvL8ESsUI%3D).

In [ ]:
Your_name = ...

### Learning from training data
A key concept in machine learning is using a subset of a dataset to train an algorithm to make estimates on a separate set of test data. The quality of the machine learning and algorithm can be assesed based on the accuracy of the predictions made on test data. Many times there are also parameters sometimes termed hyper-parameters which can be optimized through an iterative approach on test or validation data. In practice a dataset is randomly split into training and test sets using sampling. 
### k nearest neighbor
We will examine one machine learning algorithm in the laboratory, k nearest neighbor. Many of the concepts are applicable to the broad range of machine learning algorithms available.

In [ ]:
## import statements
try:
    # These lines load the tests. 
    from gofer.ok import check
    
except:
    # Install a pip package in the current Jupyter kernel
    import sys
    !{sys.executable} -m pip install git+https://github.com/grading/gradememaybe.git

    # These lines load the tests. 
    from gofer.ok import check
import numpy as np
from datascience import *
import pandas as pd
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', UserWarning)
from IPython.display import Image
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets

### k nearest  neighbor regression
We will use the k nearest neighbor algorithm to make predictions of wait time in minutes following an eruption duration ofa given number of minutes (independent variable).


In [ ]:
faithful = Table.read_table("data/faithful.csv")
faithful.scatter(0, 1, fit_line=True)

#### *Question 1* 
Use the datascience .split(n) Table method to split the dataset into 80% training and 20% test. The argument for .split(n) method,n, needs to be an integer. [See datascience documentation](https://datascience.readthedocs.io/en/master/_autosummary/datascience.tables.Table.split.html#datascience.tables.Table.split)

In [ ]:
trainf, testf = ...
print(trainf.num_rows, 'training and', testf.num_rows, 'test instances.')

In [ ]:
check('tests/q1.py')

### Nearest neighor concept
The training examines the chreacteristics of *k* nearest neighbors to the data point for which a prediction will be made. Nearness is measured using several different [metrics](https://www.nhm.uio.no/english/research/infrastructure/past/help/similarity.html) with Euclidean distance being a common one for numerical attributes.  
Euclidean distance:   
1-D $$ d(p,q) = \sqrt{(p-q)^{2}} $$   
 2-D $$ d(p,q) = \sqrt{(p_1-q_1)^{2}+(p_1-q_1)^{2}} $$
 
 For multiple points (rows):
 2-D $$ d(p,q) = \sum{{\sqrt{((p_1-q_1)^{2}+(p_1-q_1)^{2}}}} $$

###### Try different attribute values in the following 2D Euclidean distance example code below to get a feel for the computation

In [ ]:
# Example code to compute an Euclidean distance between two 2-D points
d_p_q = np.sqrt(sum((make_array(2,3)-make_array(4,3))**2))
d_p_q

To get values from Table row as an array as is done in row_distance. Note in the faithful data case we will only consider the duration column in nearest neighbor computation but in examples below we will use a 2-D array of attributes with the iris data and a 10-D array in the chemistry and molecular acidity case.

In [ ]:
f_array = np.array(faithful.row(0))
f_array

#### *Question 2* 
Define a function which is the Euclidean distance between two values. This is where we will compute the distance between two *duration* values.

In [ ]:
def distance(pt1, pt2):
    """The distance between two points, represented as arrays."""
    return ...

In [ ]:
check('tests/q2.py')

#### Rest of the nearest neighbor algorithm
Execute these cells to create the complete algorithm

In [ ]:
def row_distance(row1, row2):
    """The distance between two rows of a table."""
    return distance(np.array(row1), np.array(row2)) # Need to convert rows into arrays

def distances(training, example, output):
    """Compute the distance from example for each row in training."""
    dists = []
    attributes = training.drop(output)
    for row in attributes.rows:
        dists.append(row_distance(row, example))
    return training.with_column('Distance', dists)

def closest(training, example, k, output):
    """Return a table of the k closest neighbors to example."""
    return distances(training, example, output).sort('Distance').take(np.arange(k))

#### *Question 3*
Take an example row from the test data (testf), drop the prediction column and use the closest function to see the top 10 closest points to the target in the training data. 

In [ ]:
example_row = ...
k = ... # Number of nearest neighbors
closest(...,example_row,...,'wait')

In [ ]:
check('tests/q3.py')

#### *Question 4*
Predict the value for this row using the defined predict_nn function below and compare to the value reported for wait in the test data. How do they compare?

In [ ]:
def predict_nn(example):
    """Return the majority class among the k nearest neighbors."""
    k = 10
    return np.average(closest(trainf, example, k , 'wait').column('wait'))

In [ ]:
prediction = ...
actual = ...
print(prediction,actual)

<font color='blue'> Answer here  </font>
***  

In [ ]:
check('tests/q4.py')

#### *Question 5* Predictions
Now we will make predictions for the whole data set using the apply Table method. We will then look at the root mean squared error (RMSE) for the nearest neighbor fit and a scatter plot. Try adjusting the value of k in the predict_nn function to see it's effect on the quality of fit by rerunning these cells. Are the predicted points in a **perfect** straight line, why or why not?

In [ ]:
testf = testf.with_columns("predict",testf.apply(predict_nn,"duration"))
nn_test_predictions = testf.column("predict")
test_wait = testf.column("wait")
rmse_nn = np.mean((test_wait - nn_test_predictions) ** 2) ** 0.5

print('Test set RMSE for nearest neighbor regression:', round(rmse_nn,2))

In [ ]:
testf.scatter("duration")

<font color='blue'> Answer here  </font>
***  

### Classify iris data with machine learning
Next we will take on the problem of classifying iris data into three categories, setosa, versicolor, and virginica. Here we will also learn the basics of the k nearest neighbor algorithm.

The first data set we will look at consists of 50 samples from three species of Iris (Iris Setosa, Iris virginica, and Iris versicolor). Four features were measured including the length and the width of the sepals and petals, in centimeters for each observation.
<br><center><img src='iris.png' width=150 height=150>Iris stainglass, J.R. Smith</center>

In [ ]:
n_neighbors = 15
# Load iris data
iris = datasets.load_iris()
# We only take the first two features. 
iris_table = Table().with_columns("Name",iris.target,iris.feature_names[0],iris.data[:,0],iris.feature_names[1],iris.data[:,1])
iris_table

In [ ]:
iris.target_names

#### *Question 6*
Train and test split the iris_table @ 80% as above.

In [ ]:
train_i, test_i = ...
print(train_i.num_rows, 'training and', test_i.num_rows, 'test instances.')

In [ ]:
check('tests/q6.py')

#### *Question 7* 
With classification we need to use training data to decide how to classify data given a set of attributes, sepal length and sepal width in this case. Create a function which returns the majority classification among three possibilities in "Name" coded as 0, 1, 2 (setosa, versicolor, and virginica respectively).

In [ ]:
def majority(topkclasses):
    twos = topkclasses.where('Name', are.equal_to(2)).num_rows
    ones = ...
    zeros = ...
    # Now test to see what the majority name for each k class
    if ... and ...:
        return 2
    elif ... and ...:
        return 1
    else:
        return 0

In [ ]:
check('tests/q7.py')

In [ ]:
def classify(training, new_point, k):
    closestk = closest(training, new_point, k,"Name")
    topkclasses = closestk.select('Name')
    return majority(topkclasses)

In [ ]:
print("Prediction: ",classify(train_i,example_row,5)," Actual: ",test_i.select("Name").row(18))

In [ ]:
def predict(train, test_attributes, k):
    pred = []
    for i in np.arange(test_attributes.num_rows):
        pred.append(classify(train,test_attributes.row(i),k))
    return pred

#### *Question 8*
Make a new table called prediction which includes original columns of test Table but also includes a "predict" column.

In [ ]:
k = ...
prediction = test_i.with_columns("predict",...)
prediction.show(30)

In [ ]:
check('tests/q8.py')

### Plot decision outcomes for test set
#### *Question 9* 
Use above prediction Table to make a scatter plot of the color coded predictions based on the tweo attributes(use group="predict" in scatter plot after specifying x and y axis based on attributes).

In [ ]:
prediction.drop("Name").scatter(...,...,...)

### Fancy plot showing color coded decision boundaries
We can make a more informative plot by predicting on a grid of attribute values as shown below. Seaborn is an add-on to the Matplotlib plotting we have been using which provides more control of plotting. Execute (this may take a minute+) and study the below input and resulting output for your information.

In [ ]:
import seaborn as sns
# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
h = .1  # step size in the mesh
k = 10
x_min, x_max = iris.data[:, 0].min() - 1, iris.data[:, 0].max() + 1
y_min, y_max = iris.data[:, 1].min() - 1, iris.data[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
## Create a grid of predictions in a Table
attribute_grid = Table().with_columns(iris.feature_names[0],np.c_[xx.ravel(), yy.ravel()][:,0],iris.feature_names[1],np.c_[xx.ravel(), yy.ravel()][:,1])

Z = np.array(predict(train_i,attribute_grid,"Name",k))

# Create color maps
cmap_light = ListedColormap(['orange', 'cyan', 'cornflowerblue'])
cmap_bold = ['darkorange', 'c', 'darkblue']
# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(figsize=(8, 6))
plt.contourf(xx, yy, Z, cmap=cmap_light)

# Plot the test points but convert to numpy arrays
predictions = prediction.column('predict')
attribute1 = prediction.column(1)
attribute2 = prediction.column(2)
sns.scatterplot(x=attribute1, y=attribute2, hue=iris.target_names[predictions], palette=cmap_bold, alpha=1.0, edgecolor="black")

plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.title("3-Class classification (k = %i')"
              % (k))
plt.xlabel(iris.feature_names[0])
plt.ylabel(iris.feature_names[1])

### Use scikit learn
[Scikit learn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.neighbors) is a standard state of the art machine learning library. For demonstration purposes execute the below commands to classify  and generate a comparable output.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [ ]:
clf = neighbors.KNeighborsClassifier(k) # Initiate the classifier
x_train, x_test, y_train, y_test = train_test_split(iris.data[:,:2], iris.target, random_state=22) #  scikit split
# Now fit
clf.fit(x_train, y_train)

In [ ]:
import seaborn as sns
# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
h = .1  # step size in the mesh
x_min, x_max = iris.data[:, 0].min() - 1, iris.data[:, 0].max() + 1
y_min, y_max = iris.data[:, 1].min() - 1, iris.data[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
# Create color maps
cmap_light = ListedColormap(['orange', 'cyan', 'cornflowerblue'])
cmap_bold = ['darkorange', 'c', 'darkblue']
# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(figsize=(8, 6))
plt.contourf(xx, yy, Z, cmap=cmap_light)

# Plot also the training points
y = y_test
sns.scatterplot(x=x_test[:, 0], y=x_test[:, 1], hue=iris.target_names[y],
                    palette=cmap_bold, alpha=1.0, edgecolor="black")
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.title("3-Class classification (k = %i')"
              % (k))
plt.xlabel(iris.feature_names[0])
plt.ylabel(iris.feature_names[1])

#### *Question 10* 
Comment on the quality of the predictions by 
1. Your nearest neighjbor algorithm 
2. scikit learn 
3. Comparison

#### <font color='blue'> Answers here </font>
***

## Molecules and predicting acidity measured by pKa
Within the Jupyter notebook we can also analyze molecules and their molecular data using the library  RDKit. RDKit adds the ability to visualize 2D and 3D molecular structures. We can apply many of the data science tools we have learned to molecular data as well. First we will briefly look at acid-base chemistry and how acidity is defined. Next we will use some computed atributes of a large set of molecules to train a k nearest neighbor model to predict acidity.

#### Acid-base and pKa background
A very brief background on acid - base equilibria demonstrated for glycine. See [OpenStax Chemistry](https://openstax.org/books/chemistry-2e/pages/14-introduction) for details based on interest.
<br><center><img src='acid_base_pKa.png' width=900></center>

### RDKit
RDKit is a specialized library to handle the complexities of molecules within Python. <br><font color='red'>Unfortunately we presently can not install rdkit in our Jupyter notebook environment. You can examine the commands if interested to get a sense of the power of this library.</font>

In [ ]:
try:
    from rdkit import Chem
    from rdkit.Chem.Draw import IPythonConsole #Needed to show molecules
    from rdkit.Chem.Draw.MolDrawing import MolDrawing, DrawingOptions #Only needed if modifying defaults
    from rdkit.Chem import rdRGroupDecomposition
    from rdkit.Chem import rdDepictor
    from rdkit.Chem import PandasTools
    from rdkit.Chem import AllChem
    from rdkit.Chem import Draw
    from rdkit import DataStructs
    # Options
    DrawingOptions.bondLineWidth=1.8
    rd =True
except:
    rd = False
    print("rdkit unavailable")

##### Load detailed molecular data for 2000 molecules

In [ ]:
url = "https://raw.githubusercontent.com/robraddi/GP-SAMPL7/main/pKaDatabase/OChem/ochem0-2000.csv"
data = Table.read_table(url)
data=data.sort('N')
data.show(5)

#### *Question 11.* Select an amino acid 
 Use the Table above to view data for an amino acid of your selection from the 21 amino acids which are building blocks of protiens. See [web page](https://i.pinimg.com/originals/a2/fd/dd/a2fddd4ad8b9067bfeb0d6f51cf28e71.jpg) for possible choices. Hint: use are.containing within the .where() Table method.

In [ ]:
amino = data.where("")
amino

In [ ]:
check('tests/q11.py')

### Display molecular structure
[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system) is a shorthand language to describe molecular structure

In [ ]:
if rd:
    Chem.MolFromSmiles("[H]-O-[H]") #Water


In [ ]:
if rd:
    Chem.MolFromSmiles("C-C-O") #Ethanol

In [ ]:
if rd:
    Chem.MolFromSmiles("[NH2+]CC(O)=O") # Glycine
else:
    display(Image(url="smiles.png", width=450))

#### Selected amino acid 2D molecular structure
Try it out for fun! Use the same above syntax and the SMILES string from your above Table to display a 2D amino acid structurefrom your selection. Even if there is no rdkit, try your hand at the SMILES molecular description.

In [ ]:
smile_struct = '...'
if rd:
    Chem.MolFromSmiles(smile_struct)
else:
    print(smile_struct)

#### Code to create a grid of molecular images with labels
Execute and study the below code

In [ ]:
if rd:
    mols = [Chem.MolFromSmiles(x) for x in amino.column("SMILES") if x is not None]
    mols
    name = amino.column("NAME")

    for i,m in enumerate(mols):
       m.SetProp("Name",name[i])

    p = Draw.MolsToGridImage( [mols[x] for x in range(0,3)] , legends=[x.GetProp("Name") for x in mols],molsPerRow=3,subImgSize=(300,250), useSVG=True )
    p
else:
    print("No rdkit")

### Use pandas to add 2D structures to dataframe
We can convert our Table to pandas then use the RDKit AddMoleculeColumnToFrame method to add structures. One row has an anomolous nitrogen atom, N, so don't be alarmed by the error presented.

In [ ]:
if rd:
    df = data.to_df() # Convert Table to pandas dataframe
    PandasTools.AddMoleculeColumnToFrame(df,smilesCol='SMILES',molCol='Molecule')
    col = df.pop("Molecule")
    df.insert(0, col.name, col) # Move structure to first column
    df.head()
else:
    display(Image(url="pandas_with_chem_structures.png", width=850))

### pKa data examination
Now we will look at a data set derived from the above data but with computed molecular attributes for our machine learning. This data set is computed and described by  [Prof. Vince Voelz](http://www.voelzlab.org) in the Temple Chemistry department and a graduate student, Robert Raddi. See their paper: [Stacking Gaussian processes to improve pKa predictions in the SAMPL7 challenge](https://link.springer.com/epdf/10.1007/s10822-021-00411-8?sharing_token=yLV8dMXdxg40M_Ds_2Rhsfe4RwlQNchNByi7wbcMAY6fCl3bMLQiAhJzS2zZw-SwUkz490heLLZu1bPJ8T5LHXo1WvZkp0AJmWzXo71rszl8UaPxjqtqR-oARfxWGrTiCV0rNXy0C7IVzX6yoMYTPv2ZJfnQS-zF1pYvL8ESsUI%3D).

In [ ]:
db = pd.read_csv("data/pKaDatabase.csv")  #csv format for data
db_table = Table().from_df(db) # Datascience Table from pandas dataframe
db

In [ ]:
glycine=db_table.where("protonated microstate ID",are.containing("acetic acid"))
if rd:
    mols = [Chem.MolFromSmiles(x) for x in glycine.column("protonated microstate smiles") if x is not None]

    name = glycine.column("pKa")

    for i,m in enumerate(mols):
       m.SetProp("Name","pKa: "+str(name[i]))

    p = Draw.MolsToGridImage( [mols[x] for x in range(0,3)] , legends=[x.GetProp("Name") for x in mols],molsPerRow=2,subImgSize=(300,250), useSVG=True )
    p

### protonated microstate smiles
Place the pKa which we will predict in the first column. Use SMILES format to display structures if rdkit is available. Execute below cells.

In [ ]:
if rd:
    #PandasTools.AddMoleculeColumnToFrame(db,smilesCol='protonated microstate smiles',molCol='Molecule',includeFingerprints=True)
    PandasTools.AddMoleculeColumnToFrame(db, smilesCol='protonated microstate smiles', molCol='protonated molecule')
    PandasTools.AddMoleculeColumnToFrame(db, smilesCol='deprotonated microstate smiles', molCol='deprotonated molecule')
    col = db.pop('protonated molecule')
    db.insert(0, col.name, col)
    col = db.pop('deprotonated molecule')
    db.insert(1, col.name, col)
col = db.pop("pKa")
db.insert(0, col.name, col)
db=db.sort_values(by='pKa',ascending=False)
db.head()

In [ ]:
db[db['protonated microstate ID'].str.contains('glycine', regex=False)] # Glycine containing molecular names

### Examine a few acidity and molecular weight distribution
The below code will generate histograms for acidity as measured by pKa and molar molecular weight measured in grams per mole. Execute code and examine output.

In [ ]:
fig = plt.figure()
ax = plt.subplot(2,2,1)
ax1 = plt.subplot(2,2,2)
db.sort_values('Weight', ascending=True)
ax = db["Weight"].plot.hist(rot=0, figsize=(14, 4), bins=25, edgecolor='black', linewidth=1.2, ax=ax)
ax.set_xlabel("molecular weight", size=16)
ax.set_ylabel("", size=12)
ax.axvline(x=db['Weight'].mean(), linewidth=4, color='r')

ax1 = db["pKa"].plot.hist(rot=0, figsize=(14, 4), bins=25, edgecolor='black', linewidth=1.2, ax=ax1)#, subplots=True, layout=(2,2))
ax1.set_xlabel(r"$pK_{a}$", size=16)
ax1.set_ylabel("", size=12)
ax1.axvline(x=4, linewidth=4, color='r')
ax1.axvline(x=9, linewidth=4, color='r')
fig = ax1.get_figure()
fig.savefig("MW_dist.pdf")

### Look at pKa and molecular attribute relationships
Here we will plot some of the attributes to see if there is a relationship between their values and the pKa we are trying to predict. Execute these cells.

In [ ]:
db.plot.scatter("Weight","pKa")

In [ ]:
db.plot.scatter("AM1BCC partial charge (prot. atom)","pKa")

In [ ]:
db.plot.scatter("Bond Order","pKa")

###  Nearest Neighbor
Let's restrict our consideration to acids with 0 < pKa < 7. The reason may be evident from the distribution in the histogram of pKa's above with two peaks, one around 4 and another at 8. The negative pKa's are outliers which also will be difficult to predict. For machine learning we will also drop the 2D molecular structures (if rdkit is present).

In [ ]:
dblow = db[db["pKa"].values<7]
dblow = dblow[dblow["pKa"].values>0]

In [ ]:
if rd:
    db2=dblow.drop(['protonated molecule','deprotonated molecule'], axis = 1)
else:
    db2=dblow    

In [ ]:
molecular = Table().from_df(db2) # Now back to Table
molecular.labels # Look at labels to select correct attributes

#### Selection of attributes/features for training and prediction
We need too select the features that we will use in the training. These will include the charges computed for key atoms adjacent to the acidic proton (H+) in columns (6-11) using the AM1BCC method, ∆G_solv (kJ/mol) (prot-deprot) in column 24,solvent accessible surface area (SASA) in column 25, bond order in column 27, and Change in Enthalpy (kJ/mol) (prot-deprot) in column 28. These are the 10 features we will use. We also keep the labels and SMILES as well as the pKa we will train on.

In [ ]:
molecular=molecular.select(0,2,3,4,5,6,7,8,9,10,11,24,25,27,28)
molecular

### Train, test split
####  *Question 12* 
Split the molecular Table into train and test data using 80% for training and remembering that the split must be an integer using int() function. Again we will select certain rowsas attributes.

In [ ]:
train, test = molecular.split(...)
print(train.num_rows, 'training and', test.num_rows, 'test instances.')
train_nn = train.drop(0,2,3, 4,5, 6, 7, 8, 9, 10, 11,12,13,14)
test_nn = test.drop(0, 2,3,4, 5, 6, 7, 8, 9, 10, 11,12,13,14)
train_nn.show(3)

In [ ]:
check('tests/q12.py')

### Our k nearest neighbors code

Remember our the k nearest neighbor code from above which wewill again use here.
<code>
    def row_distance(row1, row2):
    """The distance between two rows of a table."""
    return distance(np.array(row1), np.array(row2))

def distances(training, example, output):
    """Compute the distance from example for each row in training."""
    dists = []
    attributes = training.drop(output)
    for row in attributes.rows:
        dists.append(row_distance(row, example))
    return training.with_column('Distance', dists)

def closest(training, example, k, output):
    """Return a table of the k closest neighbors to example."""
    return distances(training, example, output).sort('Distance').take(np.arange(k))
</code>

### Test algorithm
Execute these cells to define the predict_nn function for pKa, pick an example row, predict and compare.

In [ ]:
def predict_nn(example):
    """Return the majority class among the k nearest neighbors."""
    k = 10
    return np.average(closest(train_nn.drop(1,2,3,4), example, k, 'pKa').column('pKa'))

In [ ]:
# Look at closest in training set to test row. 
closest(train_nn.drop(1,2,3,4), test_nn.drop(0,1,2,3,4).row(100), k, 'pKa').select('pKa','Distance')

In [ ]:
# If we use training data in both cases we get exact match and no training, not machine learning but matching!
closest(train_nn.drop(1,2,3,4), train_nn.drop(0,1,2,3,4).row(100), k, 'pKa').select('pKa','Distance')

### Histogram of experimental acidity to be predicted
#### *Question*  
Make a histogram of acidity measured by pKa in the training data

In [ ]:
...

### *Question 13* Prediction time
Now predict the pKa of the 10 molecule in the test_nn dataset using predict. We need to drop the experimental pKa and descriptors in the first 4 columns to create and example_nn_row with the attributes for the k nearest neighbor. Discuss the quality of the fit and the name of the name of the molecule from column 1. Repeat for two more rows and discuss the prediction quality. Keep in mind that the prediction of pKa is a very challenging task for machine learning.

In [ ]:
example_nn_row = ...
print('Experimental pKa:', test_nn.column('pKa').item(100))
print('Predicted pKa using nearest neighbors:', round(predict_nn(example_nn_row),2))

In [ ]:
check('tests/q13.py')

### Now let's plot knn prediction success
Execute the next three cells

In [ ]:
exp_pKa = make_array()
predict_pKA = make_array()

In [ ]:
# This takes a while!
for i in np.arange(test_nn.num_rows):
    exp_pKa = np.append(exp_pKa,test_nn.column('pKa').item(i))
    example_nn_row = test_nn.drop(0,1,2,3,4).row(i)
    predict_pKA = np.append(predict_pKA,predict_nn(example_nn_row) )

In [ ]:
plt.scatter(exp_pKa,predict_pKA)
#calculate equation for regression line
z = np.polyfit(exp_pKa,predict_pKA, 1)
p = np.poly1d(z)
#add trendline to plot
plt.plot(exp_pKa, p(exp_pKa),'red')

### Conclusions on our k nearest neighbor model
#### *Question 14* 
Evaluate the overall quality of our machine learning prediction based on the above plot and your 3 predictions above.

<font color='blue'>Your discussion </font>
***   

### Now demonstrate knn from scikit learn
scikit learn is a standard and powerful machine learning library. Below is a demonstration for your information of the same machine learning task using scikit learn. Execute the below cells.

In [ ]:
knn = KNeighborsRegressor(n_neighbors=15, weights='distance',p=1)

In [ ]:
X = make_array()
attributes = train_nn.drop('pKa',1,2,3,4)
for i in np.arange(attributes.num_rows):
    X=np.append(X,np.array(attributes.row(i)))
X=X.reshape(attributes.num_rows,len(attributes))

In [ ]:
y=train_nn.column('pKa')
y

In [ ]:
knn.fit(X,y)

### Now test attributes

In [ ]:
attributes = test_nn.drop('pKa',1,2,3,4)
Xtest = make_array()
for i in np.arange(attributes.num_rows):
    Xtest=np.append(Xtest,np.array(attributes.row(i)))
Xtest=Xtest.reshape(attributes.num_rows,len(attributes))

In [ ]:
ytest=test_nn.column('pKa')

In [ ]:
y_predicted = knn.predict(Xtest)
predict_nn = test_nn.with_columns("pKa predict",y_predicted)

In [ ]:
plt.scatter(ytest,y_predicted)
#calculate equation for regression line
z = np.polyfit(ytest,y_predicted, 1)
p = np.poly1d(z)
# Label with equation
print(p)
#add trendline to plot
plt.plot(ytest, p(ytest),'red',label="{}".format(p))
plt.legend(fontsize="small")
plt.show()

### Return the coefficient of determination of the prediction.

The coefficient of determination $R^2$ is defined as 
 $$ (1-\frac{u}{v}) $$ u is the residual sum of squares ((y_true - y_pred)** 2).sum() and v
is the total sum of squares ((y_true - y_true.mean()) ** 2).sum(). The best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). A constant model that always predicts the expected value of y, disregarding the input features, would get a 
$R^2 = 0.0 $

In [ ]:
knn.score(Xtest, ytest) # knn score 0-1 

### Final fancy plotting of select molecules
#### *Question 15* 
Use a part of a molecular name to see if it is included in the test data and then execute the code to examine structures. Structures will be default structures if rdkit is not available.

In [ ]:
molecular_name = ...
predict_nn.where("protonated microstate ID",are.containing(molecular_name))

In [ ]:
check('tests/q15.py')

In [ ]:
if rd:
    glycine=predict_nn.where("protonated microstate ID",are.containing(molecular_name))
    mols = [Chem.MolFromSmiles(x) for x in glycine.column("protonated microstate smiles") if x is not None]
    molde = [Chem.MolFromSmiles(x) for x in glycine.column("deprotonated microstate smiles") if x is not None]
    mol = [None] * 2 * glycine.num_rows
    mol[0::2]=mols
    mol[1::2]=molde
    label = [None] * 2 * glycine.num_rows
    lpred = [None] * 2 * glycine.num_rows
    exp = glycine.column("pKa")
    pred = glycine.column("pKa predict")
    label[0::2] = exp
    label[1::2] = exp
    lpred[0::2] = pred
    lpred[1::2] = pred
    for i,m in enumerate(mol):
       m.SetProp("Name","pKa: "+str(np.round(label[i],2))+"  knn: "+str(np.round(lpred[i],2)))

    p = Draw.MolsToGridImage( [mol[x] for x in range(0,(2 * glycine.num_rows))] , legends=[x.GetProp("Name") for x in mol],molsPerRow=2,subImgSize=(300,250))
    p
else:
    display(Image(url="glycine_structures.png", width=550))

## All finished...
Run checks and submit .html and .ipynb files after downloading.

In [ ]:
# For your convenience, you can run this cell to run all the tests at once!
import glob
from gofer.ok import check
correct = 0
checks = [1,2,3,4,6,7,8,11,12,13,15]
total = len(checks)
for x in checks:
    print('Testing question {}: '.format(str(x)))
    g = check('tests/q{}.py'.format(str(x)))
    if g.grade == 1.0:
        print("Passed")
        correct += 1
    else:
        print('Failed')
        display(g)

print('Grade:  {}'.format(str(correct/total)))
print("Nice work ",Your_name)
import time;
localtime = time.asctime( time.localtime(time.time()) )
print("Submitted @ ", localtime)